#### [Bitcoin API](https://bitcoin.org/en/developer-reference#bitcoin-core-apis)

This notebook takes the [Bitcoin Developers Reference](https://bitcoin.org/en/developer-reference), specifically the [Bitcoin API](https://bitcoin.org/en/developer-reference#bitcoin-core-apis), and demonstrates the use of the Remote Procedure Calls (RPC) via python.  We employ the [regression testing mode](https://bitcoin.org/en/developer-examples#regtest-mode) of bitcoin, regtest, to start with a clean blockchain.

The notebook orders the RPC methods to illustrate how to use the API.  In the [Bitcoin Developers Reference](https://bitcoin.org/en/developer-reference) the RPC methods are in alphabetical order while the python example code below is in *discovery* sequence, starting with creating the genesis block, mining our first bitcoins, sending some to another user and showing updates to the blockchain throughout the process.

The notebook may be useful for beginners looking for a simple python interface to the Bitcoin API; we use the [python bitcoin module](https://github.com/petertodd/python-bitcoinlib) created by Peter Todd for its simplicity.  As well, the value to the user who is new to python is a simple step by step walk thru of the various methods in the API in an incremental introduction without the confusion of thousands of transactions in a typical block on mainnet.

We create two users on the regression test network (regtest):

* Bob  on RPC port 16591 
* Mary on RPC port 16592.  

We create independent peers on the same network.  

Handy shell command to see current nodes:

```ps aux |grep bitcoin```

We could add as many nodes to regtest as we like (upper limit?) and connect them in a round-robin way so that node 1 listens to node 2... and node N listens to node 1.  For our examples here, we use just two nodes in the dictionary ```connects```.

The python script below is designed for Linux (Ubuntu) and would need to be modified for other platforms.


In [1]:
#!/bin/bash
#regtest_start_network.sh
import os
import shutil

#os.system("killall --regex bitcoin.*")

idir = os.environ['HOME']+'/regtest'
if os.path.isdir(idir):
    shutil.rmtree(idir)

os.mkdir(idir)
connects = {'17591' : '17592', '17592' : '17591'}

for port in connects.keys():
    adir = idir+'/'+port
    os.mkdir(adir)
    args = " -server -txindex=1 -listen -port=" + port + \
           " -rpcuser=bitcoinrpc -rpcpassword=P0 -rpcport="+\
           str(int(port)-1000) +\
           " -datadir=" + adir + " -connect=localhost:" + connects[port] +\
           " -regtest -pid="+port+".pid -daemon -debug"
    os.system("bitcoind" + args)


Import the [python bitcoin module](https://github.com/petertodd/python-bitcoinlib) created by Peter Todd.

In [2]:
import bitcoin
import bitcoin.rpc

import bitcoin.core 
import bitcoin.core.script 

#### Connect nodes to bitcoin-rpc module

We hard-code the username and passwords here for pedagogical ease.  We display the blockchain information using the widely used, but deprecated getinfo() RPC.

getinfo() [returns](http://bitcoin.stackexchange.com/questions/18033/what-does-getinfo-output-mean):

version - The version number of this bitcoin-qt or bitcoind program itself. Both of are equivalent. -qt is simply the graphical user interface version

protocolversion: The version of the bitcoin network protocol supported by this client (user agent software).

walletversion: The version of the wallet.dat file. Wallet.dat contains bitcoin addresses and public & private key pairs for these addresses. There is additional data on the wallet. Care must be taken to not restore from an old wallet backup. New addresses generated in the wallet since the old backup was made will not exist in the old backup! [Source](https://en.bitcoin.it/wiki/Wallet)

balance: The total number of bitcoins held in the wallet.dat file.

blocks: The total number of blocks which constitute the shared block chain.

timeoffset: Seconds of difference between this node's "wall clock time" and the median reported by our network peers.

connections: the number of peers on the bitcoin P2P network that this node is connected to.

proxy: If using a proxy to connect to the network, listed here, otherwise blank.

difficulty: the current mining difficulty factor. Difficulty is increased as more miners and more hash compute power compete to be the next one to have a block of transactions added to the blockchain.

testnet: Boolean value (true OR false). There is a parallel bitcoin network, the testnet, where trials and experiments may be carried out without impacting the official, live bitcoin P2P network

keypoololdest: timestamp (UNIX epoch) of the oldest key in the keypool

keypoolsize: A number of addresses are kept in reserve by the client. This is the size of that reserve.

paytxfee: Specifies what fee the client is willing to pay to expedite transactions. Miners may choose to ignore transactions that do not pay a fee, and these fee-less transactions will have low priority on queue of pending transaction and may linger there for hours.

errors: This field may inform of different status conditions. Full list of error codes in source file [bitcoinrpc.h](https://dev.visucore.com/bitcoin/doxygen/bitcoinrpc_8h_source.html) (Examples: "Bitcoin not connected", "database error", "Keypool ran out"...) 


In [4]:
bitcoin.SelectParams('regtest')

Bob  = bitcoin.rpc.RawProxy("http://bitcoinrpc:P0@127.0.0.1:16591")
Mary = bitcoin.rpc.RawProxy("http://bitcoinrpc:P0@127.0.0.1:16592")

info = Bob.getinfo()
for key in info.keys():
    print key + ' : ' + str(info[key])

connections : 2
errors : This is a pre-release test build - use at your own risk - do not use for mining or merchant applications
blocks : 0
paytxfee : 0E-8
keypoololdest : 1442925718
walletversion : 60000
difficulty : 0E-8
testnet : False
version : 119900
proxy : 
protocolversion : 70002
timeoffset : 0
balance : 0E-8
relayfee : 0.00001000
keypoolsize : 101


#### Network attributes
The following RCPs are used to determine the network attributes.  We highlight the informational RPCs and leave the [AddNode](https://bitcoin.org/en/developer-reference#addnode) and [GetAddedNodeInfo](https://bitcoin.org/en/developer-reference#getaddednodeinfo) for later investigation.

#### Network RPCs

* [AddNode](https://bitcoin.org/en/developer-reference#addnode): attempts to add or remove a node from the addnode list, or to try a connection to a node once.
* [GetAddedNodeInfo](https://bitcoin.org/en/developer-reference#getaddednodeinfo): returns information about the given added node, or all added nodes (except onetry nodes). Only nodes which have been manually added using the addnode RPC will have their information displayed.
* [GetConnectionCount](https://bitcoin.org/en/developer-reference#GetConnectionCount): returns the number of connections to other nodes.
* [GetNetTotals](https://bitcoin.org/en/developer-reference#GetNetTotals): returns information about network traffic, including bytes in, bytes out, and the current time.
* [GetNetworkInfo](https://bitcoin.org/en/developer-reference#GetNetworkInfo): returns information about the node’s connection to the network. New in 0.9.2, Updated in 0.10.0
* [GetPeerInfo](https://bitcoin.org/en/developer-reference#GetPeerInfo): returns data about each connected network node. Updated in 0.10.0
* [Ping](https://bitcoin.org/en/developer-reference#Ping): sends a P2P ping message to all connected nodes to measure ping time. Results are provided by the getpeerinfo RPC pingtime and pingwait fields as decimal seconds. The P2P ping message is handled in a queue with all other commands, so it measures processing backlog, not just network ping.

In [ ]:
getnetworkinfo = Bob.getnetworkinfo()
print '\ngetnetworkinfo\n'
print getnetworkinfo
getpeerinfo = Bob.getpeerinfo()
print '\ngetpeerinfo\n'
print getpeerinfo
getconnectioncount = Bob.getconnectioncount()
print '\ngetconnectioncount\n'
print getconnectioncount
getnettotals = Bob.getnettotals()
print '\ngetnettotals\n'
print getnettotals


#### Block Chain RPCs

* [GetBestBlockHash](https://bitcoin.org/en/developer-reference#getbestblockhash): returns the header hash of the most recent block on the best block chain. New in 0.9.0
* [GetBlock](https://bitcoin.org/en/developer-reference#GetBlock): gets a block with a particular header hash from the local block database either as a JSON object or as a serialized block.
* [GetBlockChainInfo](https://bitcoin.org/en/developer-reference#GetBlockChainInfo): provides information about the current state of the block chain. New in 0.9.2, Updated in 0.10.0
* [GetBlockCount](https://bitcoin.org/en/developer-reference#GetBlockCount): returns the number of blocks in the local best block chain.
* [GetBlockHash](https://bitcoin.org/en/developer-reference#GetBlockHash): returns the header hash of a block at the given height in the local best block chain.
* [GetChainTips](https://bitcoin.org/en/developer-reference#GetChainTips): returns information about the highest-height block (tip) of each local block chain. New in 0.10.0
* [GetDifficulty](https://bitcoin.org/en/developer-reference#GetDifficulty): returns the proof-of-work difficulty as a multiple of the minimum difficulty.
* [GetMemPoolInfo](https://bitcoin.org/en/developer-reference#GetMemPoolInfo): returns information about the node’s current transaction memory pool. New in 0.10.0
* [GetRawMemPool](https://bitcoin.org/en/developer-reference#GetRawMemPool): returns all transaction identifiers (TXIDs) in the memory pool as a JSON array, or detailed information about each transaction in the memory pool as a JSON object.
* [GetTxOut](https://bitcoin.org/en/developer-reference#GetTxOut): returns details about a transaction output. Only unspent transaction outputs (UTXOs) are guaranteed to be available.
* [GetTxOutProof](https://bitcoin.org/en/developer-reference#GetTxOutProof): returns a hex-encoded proof that one or more specified transactions were included in a block. New in 0.11.0
* [GetTxOutSetInfo](https://bitcoin.org/en/developer-reference#GetTxOutSetInfo): returns statistics about the confirmed unspent transaction output (UTXO) set. Note that this call may take some time and that it only counts outputs from confirmed transactions—it does not count outputs from the memory pool.
* [VerifyChain](https://bitcoin.org/en/developer-reference#VerifyChain): verifies each entry in the local block chain database.
* [VerifyTxOutProof](https://bitcoin.org/en/developer-reference#VerifyTxOutProof): verifies that a proof points to one or more transactions in a block, returning the transactions the proof commits to and throwing an RPC error if the block is not in our best block chain. New in 0.11.0


In [ ]:
blockchaininfo = Bob.getblockchaininfo()
getblockcount  =  Bob.getblockcount()
getbestblockhash =  Bob.getbestblockhash()
getdifficulty  =  Bob.getdifficulty()
getchaintips   =  Bob.getchaintips()
getmempoolinfo = Bob.getmempoolinfo()

print '\nblockchaininfo\n'
print blockchaininfo
print '\ngetblockcount ' + str(getblockcount)
print '\ngetbestblockhash\n'
print getbestblockhash
print '\ngetdifficulty ' + str(getdifficulty)
print '\ngetchaintips\n'
print getchaintips
print '\ngetmempoolinfo\n'
print getmempoolinfo

print '\n\n'

bestblockhash  =  blockchaininfo['bestblockhash']
blocks         =  blockchaininfo['blocks']
print '\nblocks = ' + str(blocks)
print '\nbestblockhash = ' + str(bestblockhash)



#### Bootstrap

Bootstrap some bitcoins so we can spend them.  In regtest, you use generate and mine 101 blocks rewarding you with 50 bitcoins in a coinbase transaction.  On testnet, you use a faucet to get some bitcoins.  On mainnet, you need actual bitcoins.  We will stick with regtest for now.

In [5]:
## N.B. our balance is zero in the genesis block
print 'Initial balance, before any mining ' + str(Bob.getbalance())

Bob.generate(101)

print 'Balance after mining 101 blocks ' + str(Bob.getbalance())


Initial balance, before any mining 0E-8
Balance after mining 101 blocks 50.00000000


#### Raw Transaction RPCs

* [CreateRawTransaction](https://bitcoin.org/en/developer-reference#createrawtransaction): creates an unsigned serialized transaction that spends a previous output to a new output with a P2PKH or P2SH address. The transaction is not stored in the wallet or transmitted to the network.
* [DecodeRawTransaction](https://bitcoin.org/en/developer-reference#DecodeRawTransaction): decodes a serialized transaction hex string into a JSON object describing the transaction.
* [DecodeScript](https://bitcoin.org/en/developer-reference#DecodeScript): decodes a hex-encoded P2SH redeem script.
* [GetRawTransaction](https://bitcoin.org/en/developer-reference#GetRawTransaction): gets a hex-encoded serialized transaction or a JSON object describing the transaction. By default, Bitcoin Core only stores complete transaction data for UTXOs and your own transactions, so the RPC may fail on historic transactions unless you use the non-default txindex=1 in your Bitcoin Core startup settings.
* [SendRawTransaction](https://bitcoin.org/en/developer-reference#SendRawTransaction): validates a transaction and broadcasts it to the peer-to-peer network.
* [SignRawTransaction](https://bitcoin.org/en/developer-reference#SignRawTransaction): signs a transaction in the serialized transaction format using private keys stored in the wallet or provided in the call.

#### Generate some bitcoins with a mining start
Now let's look at the block structure, rerunning code from above.  Note that the block height is now 101.  There is one transaction for 50BTC.  Note that this is a coinbase transaction with just a vout and no vin.

How to make serialized transaction?

In [ ]:
getblockcount = Bob.getblockcount()
print '\ngetblockcount = ' + str(getblockcount)

getblockhash = Bob.getblockhash(getblockcount)
print '\ngetblockhash = ' + str(getblockhash)

print '\ngetblock\n'
getblock = Bob.getblock(getblockhash)
print getblock

tx = getblock['tx']
print '\n' + str(len(tx)) + ' Transactions\n'
print tx

for i in range(len(tx)):
    print '\nSerialized Transaction #' + str(i) +'\n'
    serializedTX = Bob.getrawtransaction(tx[i],0)
    print serializedTX
    print '\nRaw Transaction\n'
    rawTX = Bob.getrawtransaction(tx[i],1)
    print rawTX
    print '\nDecoded Transaction\n'
    decodedTX    = Bob.decoderawtransaction(serializedTX)
    print decodedTX


Now that we have a raw transaction, let's look at the details.  As noted, the first 100 blocks in regtest are blank, necessary for mining our first coinbase reward of 50BTC.  This shows in block 101 with a single transaction, denoted rawTX below.  rawTX is a JSON object with:

1. blockhash from block 101
2. vout an array (len=1) of scriptPubKey, a JSON object of reqSigs and a value (50BTC here)
3. vin an array (len=1) with a key called "coinbase", some value and a sequence number

We veirfy that the serialized transaction hex is the same as the "hex" entry in the transaction.  We note that the txid is repeated in the raw transaction.  We also note that the confirmations = 1 (no further mining has been done on this block), that there is a single transaction (the mining of 50btc) and the [blocktime](https://en.bitcoin.it/wiki/Block_timestamp) and time are the same, while the locktime is the beginning of the epoch.  

Note that an address has been created, in general an array of addresses, each corresponding to a transaction output (vout).  Here, a new address has been created corresponding to the miner's address for the 50BTC reward.

In [ ]:
from datetime import datetime as dt
#import pytz

print rawTX 
print '\n\n'

print 'blockhash = ' + str(rawTX['blockhash']) + '\n'
for i in range(len(rawTX['vout'])):
    spk = rawTX['vout'][i]['scriptPubKey']
    print 'vout ' + str(i) + ' : ' + str(spk) + '\n'
    for field in spk.keys(): 
        #['reqSigs','hex','addresses','asm','type']:
        print 'vout ' + str(i) + ' ' + field + '  : ' + str(spk[field])
    print 'vout ' + str(i) + ' value : ' + str(rawTX['vout'][i]['value'])
    print 'vout ' + str(i) + ' n     : ' + str(rawTX['vout'][i]['n'])

print '\nserialized hex = ' + str(rawTX['hex'])
print 'Is serialized hex == rawTX["hex"]? ' + str(rawTX['hex']==serializedTX) + '\n'

for i in range(len(rawTX['vin'])):
    spk = rawTX['vin'][i]
    print 'vin ' + str(i) + ' : ' + str(spk) + '\n'
    for field in spk.keys(): 
        #['reqSigs','hex','addresses','asm','type']:
        print 'vin ' + str(i) + ' ' + field + '  : ' + str(spk[field])

print '\n'
for field in ['txid','blocktime','version','confirmations','time','locktime']:
    if field in ['blocktime','time','locktime']:
        print field + ' = ' + str(rawTX[field]) +\
        ' ' + dt.fromtimestamp(rawTX[field]).strftime('%Y-%m-%d:%H%M%S')
    else:
        print field + ' = ' + str(rawTX[field])


In [ ]:
##The P2PKH or P2SH addresses used in this transaction, 
##or the computed P2PKH address of any pubkeys in this transaction. 

##N.B. the developer's guide states that the address is a "A P2PKH or P2SH address"
##but we see this initiate transactions as a type="pubkey".  Should check on mainnet.

addresses = rawTX['vout'][0]['scriptPubKey']['addresses']
print addresses
for address in addresses:
    print address
    print Bob.getbalance()
    print Bob.getreceivedbyaddress(address,0)
print Bob.listreceivedbyaddress(0)

Let's create an address and send some bitcoins there.  Some things to notice:

0.  We set up a new proxy, using a different RPC port on regtest
1.  Address is *NOT* on blockchain, it is just a 32 byte hash that is almost certainly unique and never before used.
2.  We send some coins to new address, note the transaction structure
3.  The mempool shows the pending transaction
4.  We need to mine to write the transaction to the blockchain

In [6]:
print 'Mary\'s balance = ' + str(Mary.getbalance())
#print 'Mary\'s peers: '
#print Mary.getpeerinfo()

getnewaddress = Mary.getnewaddress()
print '\nNew address ' + str(getnewaddress)
print '\nMary\'s address has received how many BTC? ' +\
str(Mary.getreceivedbyaddress(getnewaddress,0))

##have Bob (proxy) send 25 bitcoins to Mary
txid = Bob.sendtoaddress(getnewaddress,25)


Mary's balance = 0E-8

New address mmT3ER6w98jZAKwtTZrr3DSxrchS7fGxKW

Mary's address has received how many BTC? 0E-8


In [7]:
getmempoolinfo = Bob.getmempoolinfo()
getrawmempool = Bob.getrawmempool(True)
print '\ngetmempoolinfo ' + str(getmempoolinfo)
print '\ngetrawmempool'
print getrawmempool
print '\n'
for key in getrawmempool.keys():
    for field in getrawmempool[key].keys():
        print str(field) + ' : ' + str(getrawmempool[key][field])
#print '\ntxid from sendtoaddress output ' + str(txid)
print '\nIs the send txid the same as memory pool txid? ****' +\
    str(txid == getrawmempool.keys()[0]) + '****'

print '\nMary\'s balance before mining = ' + str(Mary.getbalance())
print 'Bob\'s balance before mining = ' + str(Bob.getbalance())

##how can I see transaction details before mining?
print '\nMemory Pool Raw Transaction Data\n'
import pprint
pprint.pprint(Bob.getrawtransaction(txid,1))

##N.B. no transaction on the blockchain yet!!!



getmempoolinfo {u'bytes': 191, u'size': 1}

getrawmempool
{u'f58f06ca1920f17abab651ebce5e43a6e778aa0707e105dd767427d624b50d4e': {u'fee': Decimal('0.00000192'), u'startingpriority': Decimal('6410256410.25641060'), u'height': 101, u'depends': [], u'time': 1442932681, u'currentpriority': Decimal('6410256410.25641060'), u'size': 191}}


fee : 0.00000192
startingpriority : 6410256410.25641060
height : 101
depends : []
time : 1442932681
currentpriority : 6410256410.25641060
size : 191

Is the send txid the same as memory pool txid? ****True****

Mary's balance before mining = 0E-8
Bob's balance before mining = 24.99999808

Memory Pool Raw Transaction Data

{u'hex': u'01000000015bfebe850b91263319767b717c2205356fdf1b67f27943f0f22f9f237eca777400000000484730440220720f12367d57646dc2a59f29a3d1b4a346307097ec5f8462570b7ea4c17441ec022019313105ed963f19ebe9b70887abb92ca0850d2d57553320805f76960bc4190801feffffff0240f80295000000001976a9147ac57c35681d0269ec00602470056d9b72d32dd788ac00f90295000000001976a91

Bob mines 6 more blocks, after the fifth, the 25BTC sent to Mary is confirmed and shows in her balance.  Note that the transaction created in block 101, when Bob sent Mary 25BTC shows up in the first block mined, but the balance isn't updated until some number of blocks have been processed.  How is this determined????

In [8]:
for i in range(7):
    Bob.generate(1)
    getblockcount   =  Bob.getblockcount()
    getblockhash   =  Bob.getblockhash(getblockcount)
    getblock       =  Bob.getblock(getblockhash)
    print 'Block #' + str(getblockcount) + ' Mary\'s balance ' + str(Mary.getbalance())
    print 'txids ' + str(getblock['tx'])
    

Block #102 Mary's balance 0E-8
txids [u'1d8d53a8c305b1a5c71eff3dbe5f6a08df1c4d7f96293ecf314f208f82b426a3', u'f58f06ca1920f17abab651ebce5e43a6e778aa0707e105dd767427d624b50d4e']
Block #103 Mary's balance 0E-8
txids [u'6ed8aacc17f937782a72d5d7ce3fa6a398af0c6a61431e90dc782ac2d2b698fa']
Block #104 Mary's balance 0E-8
txids [u'f694da34fe6eaf531d391c14ae2725cd03a6deb282bae96e00710b9dc4c9d666']
Block #105 Mary's balance 25.00000000
txids [u'a83120ef1edb1c9e9c4202643ccceb6fd21dd84db942aa0d3424ba382340707f']
Block #106 Mary's balance 25.00000000
txids [u'9a2c9006abdb6d97aeb1465c9d4cc9bc7535dc1d05339d5eb0b7019541de6f35']
Block #107 Mary's balance 25.00000000
txids [u'4bb620b512a5cda8ba0efd5bc0575f252b821f4e17148ec7e68f54d0c2bf3672']
Block #108 Mary's balance 25.00000000
txids [u'ece9cf5633cc8c04f1d830bc5b6c51e73588c25d9a808f5f84c24beff461fd7b']


In [ ]:
print ' Mary\'s balance ' + str(Mary.getbalance())

### Wallet RPCs

Note: the wallet RPCs are only available if Bitcoin Core was built with wallet support, which is the default.

* [AddMultiSigAddress](https://bitcoin.org/en/developer-reference#AddMultiSigAddress): adds a P2SH multisig address to the wallet.
* [BackupWallet](https://bitcoin.org/en/developer-reference#BackupWallet): safely copies wallet.dat to the specified file, which can be a directory or a path with filename.
* [DumpPrivKey](https://bitcoin.org/en/developer-reference#DumpPrivKey): returns the wallet-import-format (WIP) private key corresponding to an address. (But does not remove it from the wallet.)
* [DumpWallet](https://bitcoin.org/en/developer-reference#DumpWallet): creates or overwrites a file with all wallet keys in a human-readable format.
* [EncryptWallet](https://bitcoin.org/en/developer-reference#EncryptWallet): encrypts the wallet with a passphrase. This is only to enable encryption for the first time. After encryption is enabled, you will need to enter the passphrase to use private keys.
* [GetAccountAddress](https://bitcoin.org/en/developer-reference#GetAccountAddress): returns the current Bitcoin address for receiving payments to this account. If the account doesn’t exist, it creates both the account and a new address for receiving payment. Once a payment has been received to an address, future calls to this RPC for the same account will return a different address.
* [GetAccount](https://bitcoin.org/en/developer-reference#GetAccount): returns the name of the account associated with the given address.
* [GetAddressesByAccount](https://bitcoin.org/en/developer-reference#GetAddressesByAccount): returns a list of every address assigned to a particular account.
* [GetBalance](https://bitcoin.org/en/developer-reference#GetBalance): gets the balance in decimal bitcoins across all accounts or for a particular account.
* [GetNewAddress](https://bitcoin.org/en/developer-reference#GetNewAddress): returns a new Bitcoin address for receiving payments. If an account is specified, payments received with the address will be credited to that account.
* [GetRawChangeAddress](https://bitcoin.org/en/developer-reference#GetRawChangeAddress): returns a new Bitcoin address for receiving change. This is for use with raw transactions, not normal use.
* [GetReceivedByAccount](https://bitcoin.org/en/developer-reference#GetReceivedByAccount): returns the total amount received by addresses in a particular account from transactions with the specified number of confirmations. It does not count coinbase transactions.
* [GetReceivedByAddress](https://bitcoin.org/en/developer-reference#GetReceivedByAddress): returns the total amount received by the specified address in transactions with the specified number of confirmations. It does not count coinbase transactions.
* [GetTransaction](https://bitcoin.org/en/developer-reference#GetTransaction): gets detailed information about an in-wallet transaction. Updated in 0.10.0
* [GetUnconfirmedBalance](https://bitcoin.org/en/developer-reference#GetUnconfirmedBalance): returns the wallet’s total unconfirmed balance.
* [GetWalletInfo](https://bitcoin.org/en/developer-reference#GetWalletInfo): provides information about the wallet. New in 0.9.2
* [ImportAddress](https://bitcoin.org/en/developer-reference#ImportAddress): adds an address or pubkey script to the wallet without the associated private key, allowing you to watch for transactions affecting that address or pubkey script without being able to spend any of its outputs. New in 0.10.0
* [ImportPrivKey](https://bitcoin.org/en/developer-reference#ImportPrivKey): adds a private key to your wallet. The key should be formatted in the wallet import format created by the dumpprivkey RPC.
* [ImportWallet](https://bitcoin.org/en/developer-reference#ImportWallet): imports private keys from a file in wallet dump file format (see the dumpwallet RPC). These keys will be added to the keys currently in the wallet. This call may need to rescan all or parts of the block chain for transactions affecting the newly-added keys, which may take several minutes.
* [KeyPoolRefill](https://bitcoin.org/en/developer-reference#KeyPoolRefill): fills the cache of unused pre-generated keys (the keypool).
* [ListAccounts](https://bitcoin.org/en/developer-reference#ListAccounts): lists accounts and their balances. Updated in 0.10.0
* [ListAddressGroupings](https://bitcoin.org/en/developer-reference#ListAddressGroupings): lists groups of addresses that may have had their common ownership made public by common use as inputs in the same transaction or from being used as change from a previous transaction.
* [ListLockUnspent](https://bitcoin.org/en/developer-reference#ListLockUnspent): returns a list of temporarily unspendable (locked) outputs.
* [ListReceivedByAccount](https://bitcoin.org/en/developer-reference#ListReceivedByAccount): lists the total number of bitcoins received by each account. Updated in 0.10.0
* [ListReceivedByAddress](https://bitcoin.org/en/developer-reference#ListReceivedByAddress): lists the total number of bitcoins received by each address. Updated in 0.10.0
* [ListSinceBlock](https://bitcoin.org/en/developer-reference#ListSinceBlock): gets all transactions affecting the wallet which have occurred since a particular block, plus the header hash of a block at a particular depth. Updated in 0.10.0
* [ListTransactions](https://bitcoin.org/en/developer-reference#ListTransactions): returns the most recent transactions that affect the wallet. Updated in 0.10.0
* [ListUnspent](https://bitcoin.org/en/developer-reference#ListUnspent): returns an array of unspent transaction outputs belonging to this wallet. Updated in 0.10.0
* [LockUnspent](https://bitcoin.org/en/developer-reference#LockUnspent): temporarily locks or unlocks specified transaction outputs. A locked transaction output will not be chosen by automatic coin selection when spending bitcoins. Locks are stored in memory only, so nodes start with zero locked outputs and the locked output list is always cleared when a node stops or fails.
* [Move](https://bitcoin.org/en/developer-reference#Move): moves a specified amount from one account in your wallet to another using an off-block-chain transaction.
* [SendFrom](https://bitcoin.org/en/developer-reference#SendFrom): spends an amount from a local account to a bitcoin address.
* [SendMany](https://bitcoin.org/en/developer-reference#SendMany): creates and broadcasts a transaction which sends outputs to multiple addresses.
* [SendToAddress](https://bitcoin.org/en/developer-reference#SendToAddress): spends an amount to a given address.
* [SetAccount](https://bitcoin.org/en/developer-reference#SetAccount): puts the specified address in the given account.
* [SetTxFee](https://bitcoin.org/en/developer-reference#SetTxFee): sets the transaction fee per kilobyte paid by transactions created by this wallet.
* [SignMessage](https://bitcoin.org/en/developer-reference#SignMessage): signs a message with the private key of an address.
* [WalletLock](https://bitcoin.org/en/developer-reference#WalletLock): removes the wallet encryption key from memory, locking the wallet. After calling this method, you will need to call walletpassphrase again before being able to call any methods which require the wallet to be unlocked.
* [WalletPassphrase](https://bitcoin.org/en/developer-reference#WalletPassphrase): stores the wallet decryption key in memory for the indicated number of seconds. Issuing the walletpassphrase command while the wallet is already unlocked will set a new unlock time that overrides the old one.
* [WalletPassphraseChange](https://bitcoin.org/en/developer-reference#walletpassphrasechange): changes the wallet passphrase from ‘old passphrase’ to ‘new passphrase’.



In [15]:
print '\nBob\'s Wallet\n'
wallet= Bob.getwalletinfo()
for key in wallet.keys():
    print key + '\t' + str(wallet[key])
print '\nMary\'s Wallet\n'
wallet= Mary.getwalletinfo()
for key in wallet.keys():
    print key + '\t' + str(wallet[key])


Bob's Wallet

unconfirmed_balance	0E-8
paytxfee	0E-8
keypoololdest	1442925718
walletversion	60000
immature_balance	5000.00000192
txcount	109
balance	374.99999808
keypoolsize	101

Mary's Wallet

unconfirmed_balance	0E-8
paytxfee	0E-8
keypoololdest	1442925718
walletversion	60000
immature_balance	0E-8
txcount	1
balance	25.00000000
keypoolsize	101


[{u'category': u'receive', u'account': u'', u'blockhash': u'4fe9011aba9053ef0079094f644c2f1e7a24650833145e477b7e735e0c98d776', u'vout': 1, u'blockindex': 1, u'walletconflicts': [], u'time': 1442932681, u'txid': u'f58f06ca1920f17abab651ebce5e43a6e778aa0707e105dd767427d624b50d4e', u'blocktime': 1442932701, u'amount': Decimal('25.00000000'), u'confirmations': 7, u'timereceived': 1442932681, u'address': u'mmT3ER6w98jZAKwtTZrr3DSxrchS7fGxKW'}]


In [25]:
import time
print '\nMary has ' + str(len(Mary.listtransactions())) + ' transactions from Bob\'s largesse'

print '\nMary\'s first address has received how many BTC? ' +\
str(Mary.getreceivedbyaddress( Mary.listtransactions()[0]['address'],0))
#str(Mary.getreceivedbyaddress('mmT3ER6w98jZAKwtTZrr3DSxrchS7fGxKW',0))

print '\nBob has ' + str(len(Bob.listtransactions())) + ' transactions from all that mining'
#print Bob.listtransactions()




##let's send Mary some more of Bob's bitcoins so we can see her unconfirmed balance
getnewaddress = Mary.getnewaddress()
print '\nNew address ' + str(getnewaddress)

##have Bob (proxy) send 0.5 bitcoins to Mary
txid = Bob.sendtoaddress(getnewaddress,0.5)

time.sleep(2)

print '\nMary\'s unconfirmed balance ' + str(Mary.getunconfirmedbalance())
print '\nMary\'s confirmed balance ' + str(Mary.getbalance())

##let's mine 6 blocks
Bob.generate(6)

time.sleep(2)

print 'After Bob\'s mining'
print '\nMary\'s unconfirmed balance ' + str(Mary.getunconfirmedbalance())
print '\nMary\'s confirmed balance ' + str(Mary.getbalance())



Mary has 5 transactions from Bob's largesse

Mary's first address has received how many BTC? 25.00000000

Bob has 10 transactions from all that mining

New address mzvyfDiqtpmkVNzGLTd19KvJvzxpYLmn8u

Mary's unconfirmed balance 2.50000000

Mary's confirmed balance 25.00000000
After Bob's mining

Mary's unconfirmed balance 0E-8

Mary's confirmed balance 27.50000000


#### Wallet Programs

Permitting receiving and spending of satoshis is the only essential feature of wallet software—but a particular wallet program doesn’t need to do both things. Two wallet programs can work together, one program distributing public keys in order to receive satoshis and another program signing transactions spending those satoshis.

Wallet programs also need to interact with the peer-to-peer network to get information from the block chain and to broadcast new transactions. However, the programs which distribute public keys or sign transactions don’t need to interact with the peer-to-peer network themselves.

This leaves us with three necessary, but separable, parts of a wallet system: a public key distribution program, a signing program, and a networked program. In the subsections below, we will describe common combinations of these parts.

#### Full-Service Wallets

The simplest wallet is a program which performs all three functions: it generates private keys, derives the corresponding public keys, helps distribute those public keys as necessary, monitors for outputs spent to those public keys, creates and signs transactions spending those outputs, and broadcasts the signed transactions.

![](https://bitcoin.org/img/dev/en-wallets-full-service.svg "Full Service Wallets")

Full-Service Wallets

As of this writing, almost all popular wallets can be used as full-service wallets.

The main advantage of full-service wallets is that they are easy to use. A single program does everything the user needs to receive and spend satoshis.

The main disadvantage of full-service wallets is that they store the private keys on a device connected to the Internet. The compromise of such devices is a common occurrence, and an Internet connection makes it easy to transmit private keys from a compromised device to an attacker.

### Utility RPCs

* [CreateMultiSig](https://bitcoin.org/en/developer-reference#CreateMultiSig): creates a P2SH multi-signature address.
* [EstimateFee](https://bitcoin.org/en/developer-reference#EstimateFee): estimates the transaction fee per kilobyte that needs to be paid for a transaction to be included within a certain number of blocks. New in 0.10.0
* [EstimatePriority](https://bitcoin.org/en/developer-reference#EstimatePriority): estimates the priority that a transaction needs in order to be included within a certain number of blocks as a free high-priority transaction. New in 0.10.0
* [ValidateAddress](https://bitcoin.org/en/developer-reference#ValidateAddress): returns information about the given Bitcoin address.
* [VerifyMessage](https://bitcoin.org/en/developer-reference#VerifyMessage): verifies a signed message.

This code determines whether a transaction is included in a Merkle Tree.  From [Luke Dashjr](https://github.com/Crypto-Expert/stratum-mining/blob/master/lib/merkletree.py)

In [ ]:
# Eloipool - Python Bitcoin pool server
# Copyright (C) 2011-2012  Luke Dashjr <luke-jr+eloipool@utopios.org>
#
# This program is free software: you can redistribute it and/or modify
# it under the terms of the GNU Affero General Public License as
# published by the Free Software Foundation, either version 3 of the
# License, or (at your option) any later version.
#
# This program is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU Affero General Public License for more details.
#
# You should have received a copy of the GNU Affero General Public License
# along with this program.  If not, see <http://www.gnu.org/licenses/>.

# original code https://github.com/Crypto-Expert/stratum-mining/blob/master/lib/merkletree.py
# http://code.runnable.com/U3jqtyYUmAUxtsSS/bitcoin-block-merkle-root-python

from hashlib import sha256
from util import doublesha

class MerkleTree:
    def __init__(self, data, detailed=False):
        self.data = data
        self.recalculate(detailed)
        self._hash_steps = None
    
    def recalculate(self, detailed=False):
        L = self.data
        steps = []
        if detailed:
            detail = []
            PreL = []
            StartL = 0
        else:
            detail = None
            PreL = [None]
            StartL = 2
        Ll = len(L)
        if detailed or Ll > 1:
            while True:
                if detailed:
                    detail += L
                if Ll == 1:
                    break
                steps.append(L[1])
                if Ll % 2:
                    L += [L[-1]]
                L = PreL + [doublesha(L[i] + L[i + 1]) for i in range(StartL, Ll, 2)]
                Ll = len(L)
        self._steps = steps
        self.detail = detail
    
    def hash_steps(self):
        if self._hash_steps == None:
            self._hash_steps = doublesha(''.join(self._steps))
        return self._hash_steps
        
    def withFirst(self, f):
        steps = self._steps
        for s in steps:
            f = doublesha(f + s)
        return f
    
    def merkleRoot(self):
        return self.withFirst(self.data[0])

# MerkleTree tests
def _test():
    import binascii
    import time    
        
    mt = MerkleTree([None] + [binascii.unhexlify(a) for a in [
        '999d2c8bb6bda0bf784d9ebeb631d711dbbbfe1bc006ea13d6ad0d6a2649a971',
        '3f92594d5a3d7b4df29d7dd7c46a0dac39a96e751ba0fc9bab5435ea5e22a19d',
        'a5633f03855f541d8e60a6340fc491d49709dc821f3acb571956a856637adcb6',
        '28d97c850eaf917a4c76c02474b05b70a197eaefb468d21c22ed110afe8ec9e0',
    ]])
    assert(
        b'82293f182d5db07d08acf334a5a907012bbb9990851557ac0ec028116081bd5a' ==
        binascii.b2a_hex(mt.withFirst(binascii.unhexlify('d43b669fb42cfa84695b844c0402d410213faa4f3e66cb7248f688ff19d5e5f7')))
    )
    
    print '82293f182d5db07d08acf334a5a907012bbb9990851557ac0ec028116081bd5a'
    txes = [binascii.unhexlify(a) for a in [
        'd43b669fb42cfa84695b844c0402d410213faa4f3e66cb7248f688ff19d5e5f7',
        '999d2c8bb6bda0bf784d9ebeb631d711dbbbfe1bc006ea13d6ad0d6a2649a971',
        '3f92594d5a3d7b4df29d7dd7c46a0dac39a96e751ba0fc9bab5435ea5e22a19d',
        'a5633f03855f541d8e60a6340fc491d49709dc821f3acb571956a856637adcb6',
        '28d97c850eaf917a4c76c02474b05b70a197eaefb468d21c22ed110afe8ec9e0',
    ]]
             
    s = time.time()
    mt = MerkleTree(txes)
    for x in range(100):
        y = int('d43b669fb42cfa84695b844c0402d410213faa4f3e66cb7248f688ff19d5e5f7', 16)
        #y += x
        coinbasehash = binascii.unhexlify("%x" % y)
        x = binascii.b2a_hex(mt.withFirst(coinbasehash))

    print x
    print time.time() - s

if __name__ == '__main__':
    _test()
    

This code snippet illustrates a simplified proof-of-work algorithm *not* used by miners; by incementing the nonce and trying difficulties from 1-31 bits (2 - 2^32).  Note that this runs for a long time, more than 20 minutes on a 4GB-RAM Ubuntu box.

From [Mastering Bitcoin](http://chimera.labs.oreilly.com/books/1234000001802/ch08.html#_proof_of_work_algorithm)

In [ ]:
#!/usr/bin/env python
# example of proof-of-work algorithm

import hashlib
import time

max_nonce = 2 ** 32 # 4 billion

def proof_of_work(header, difficulty_bits):

    # calculate the difficulty target
    target = 2 ** (256-difficulty_bits)

    for nonce in xrange(max_nonce):
        hash_result = hashlib.sha256(str(header)+str(nonce)).hexdigest()

        # check if this is a valid result, below the target
        if long(hash_result, 16) < target:
            print "Success with nonce %d" % nonce
            print "Hash is %s" % hash_result
            return (hash_result,nonce)

    print "Failed after %d (max_nonce) tries" % nonce
    return nonce


if __name__ == '__main__':

    nonce = 0
    hash_result = ''

    # difficulty from 0 to 31 bits
    for difficulty_bits in xrange(32):

        difficulty = 2 ** difficulty_bits
        print "Difficulty: %ld (%d bits)" % (difficulty, difficulty_bits)

        print "Starting search..."

        # checkpoint the current time
        start_time = time.time()

        # make a new block which includes the hash from the previous block
        # we fake a block of transactions - just a string
        new_block = 'test block with transactions' + hash_result

        # find a valid nonce for the new block
        (hash_result, nonce) = proof_of_work(new_block, difficulty_bits)

        # checkpoint how long it took to find a result
        end_time = time.time()

        elapsed_time = end_time - start_time
        print "Elapsed Time: %.4f seconds" % elapsed_time

        if elapsed_time > 0:

            # estimate the hashes per second
            hash_power = float(long(nonce)/elapsed_time)
            print "Hashing Power: %ld hashes per second" % hash_power